# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
imported_cities_df = pd.read_csv('output_data/cities.csv').reset_index(drop=True)
imported_cities_df.head()

imported_cities_df.columns

cleaned_cities = imported_cities_df.drop(['Unnamed: 0'], axis=1)
cleaned_cities.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Rikitea,-23.12,-134.97,77.18,65,1,2.51,PF
1,Saint-Pierre,-21.34,55.48,77.00,73,9,17.22,RE
2,Jamestown,42.10,-79.24,33.01,92,90,5.82,US
3,Puteyets,65.17,57.09,32.72,92,68,8.03,RU
4,Piaçabuçu,-10.41,-36.43,80.89,81,6,8.05,BR
5,Shache,38.42,77.24,53.19,60,100,1.86,CN
6,Clyde River,70.47,-68.59,10.40,85,40,18.34,CA
7,Chapleau,47.84,-83.40,35.60,37,75,8.05,CA
8,Ossora,59.24,163.07,19.98,91,0,7.18,RU
9,Chui,-33.70,-53.46,64.42,77,2,6.64,UY


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
gmaps.configure(api_key=g_key)

In [27]:
locations = cleaned_cities[["Lat", "Lng"]].astype(float)
humidity_rate = cleaned_cities["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, dissipating=False, max_intensity=250, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
ideal_cities = cleaned_cities.loc[(cleaned_cities['Max Temp'] < 70) & (cleaned_cities['Max Temp'] > 50) & (cleaned_cities['Wind Speed'] < 5) & (cleaned_cities['Cloudiness'] > 50)]
ideal_cities_final = ideal_cities.reset_index()
ideal_cities_final.head(100)

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,5,Shache,38.42,77.24,53.19,60,100,1.86,CN
1,19,Bredasdorp,-34.53,20.04,55.40,76,100,3.36,ZA
2,44,Santa Paula,34.35,-119.06,66.00,58,73,3.00,US
3,99,Collie,-33.37,116.15,54.00,59,86,1.59,AU
4,147,Kruisfontein,-34.00,24.73,59.94,83,90,3.96,ZA
5,162,Menongue,-14.66,17.69,67.05,50,74,4.45,AO
6,209,Ngunguru,-35.62,174.50,62.01,97,58,1.01,NZ
7,228,Ji’an,27.12,114.98,66.94,86,63,4.36,CN
8,234,Rāvar,31.27,56.81,63.18,34,85,1.61,IR
9,240,Guérande,47.33,-2.43,59.00,93,55,3.36,FR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [37]:
hotel_df = ideal_cities_final.copy()
hotel_df['Hotel Name'] = ''

In [50]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params).json()
    
    #print(name_address)

#   print(json.dumps(name_address, indent=4, sort_keys=True))


    try:
        print(f'Retrieving info for nearest hotel to {lat}, {lng}...')
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing Data")

Retrieving info for nearest hotel to 38.42, 77.24...
Retrieving info for nearest hotel to -34.53, 20.04...
Retrieving info for nearest hotel to 34.35, -119.06...
Retrieving info for nearest hotel to -33.37, 116.15...
Retrieving info for nearest hotel to -34.0, 24.73...
Missing Data
Retrieving info for nearest hotel to -14.66, 17.69...
Retrieving info for nearest hotel to -35.62, 174.5...
Retrieving info for nearest hotel to 27.12, 114.98...
Retrieving info for nearest hotel to 31.27, 56.81...
Retrieving info for nearest hotel to 47.33, -2.43...
Retrieving info for nearest hotel to -41.87, -73.82...
Retrieving info for nearest hotel to 3.52, 39.06...
Retrieving info for nearest hotel to -2.47, 28.82...
Retrieving info for nearest hotel to 29.11, 58.36...
Retrieving info for nearest hotel to -45.58, -72.07...
Retrieving info for nearest hotel to -45.87, 170.5...
Retrieving info for nearest hotel to 27.37, 87.2...
Retrieving info for nearest hotel to 44.27, 9.4...
Retrieving info for near

In [64]:
hotel_df.head(100)
cleaned_hotel_df = hotel_df.reset_index(drop=True)
cleaned_hotel_df.drop(['index'], axis=1)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
0,Shache,38.42,77.24,53.19,60,100,1.86,CN,New Century Hotel
1,Bredasdorp,-34.53,20.04,55.40,76,100,3.36,ZA,Bredasdorp Country Manor
2,Santa Paula,34.35,-119.06,66.00,58,73,3.00,US,Glen Tavern Inn
3,Collie,-33.37,116.15,54.00,59,86,1.59,AU,Banksia Motel
4,Kruisfontein,-34.00,24.73,59.94,83,90,3.96,ZA,
5,Menongue,-14.66,17.69,67.05,50,74,4.45,AO,Ritz Lauca
6,Ngunguru,-35.62,174.50,62.01,97,58,1.01,NZ,Quality Hotel Oceans Tutukaka
7,Ji’an,27.12,114.98,66.94,86,63,4.36,CN,Ji'an Hotel
8,Rāvar,31.27,56.81,63.18,34,85,1.61,IR,Hotel sperlous
9,Guérande,47.33,-2.43,59.00,93,55,3.36,FR,Villa Escoublac


In [65]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in cleaned_hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [75]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations,
    info_box_content=hotel_info)

fig = gmaps.figure()

fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)
    
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))